# Example Question

Example 4.1. Suppose a wheel is rotating on an axle, the total moment of inirtia being $J$, 
and it is required to bring the system to rest by applying a braking torque $u(t)$. 
The equation of motion is

\begin{equation}
    J\frac{dx}{dt} = u 
\end{equation}



## Method

 Now take $ t \in [0,1] $ and B.C. are $ x(0) = 10 $ and $ x(1) = 0 $. To solve this problem we will construct trail solution for $x$ and for $u$ with different Neural Networks for each of them and then optimize them simultaneously. 

ODE: 

\begin{equation}
    \frac{dx}{dt} = \frac{u(t)}{J}
\end{equation}


Where

- $ x(t) $ is angular velocity.
- $ J $ is total moment of inertia.
- $ u(t) $ is breaking torque.


Let

\begin{equation}
    \begin{aligned}
        t &\in [0,1] \\
        J &= 10 \\
        x(0) &= 10 \\
        x(1) &= 0
    \end{aligned}
\end{equation}


Exact (actual) solution of above ODE is:

\begin{equation}
    u_a(t_0) = \frac{-J x(t_0)}{t_1-t_0}
\end{equation}

for above  condtions,

\begin{equation}
    \begin{aligned}
        u_a(0) &= -100 \\
        u_a(1) &= 0
    \end{aligned}
\end{equation}


Trail solution is of the form:

\begin{equation}
    x(t)_{tr} = A(1-t) + Bt + t (1-t) \frac{1}{J} u(t)_{tr} + t (1-t) N_x(t,p) 
\end{equation}

\begin{equation}
    u(t)_{tr} = (1-t) C + t N_u(t,p)
\end{equation}

Where,

\begin{equation}
    \begin{aligned}
        A &= x(0) = 10 \\
        B &= x(1) = 0 \\
        C &= u_a(0) = -100 \\
    \end{aligned}
    \tag{5}
\end{equation}

Therefore we can write,

\begin{equation}
    x(t)_{tr} = \frac{(1-t)}{J}[J A + t[(1-t) C + t N_u + J N_x]]
\end{equation}

So for the above equation the Loss can be calculated using following equation,

\begin{equation}
    L = \sum \left[ J\frac{d x(t)_{tr}}{dt} - u_{tr} \right]
\end{equation}





In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats("svg")
from tqdm import tqdm

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [27]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.Hidden_layer = nn.Linear(in_features=1, out_features=10)
        self.Output_layer = nn.Linear(in_features=10, out_features=1, bias=False)
        self.Sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.Hidden_layer(x)
        x = self.Sigmoid(x)
        output = self.Output_layer(x)
        return output

In [28]:
NeuralNetwork()

NeuralNetwork(
  (Hidden_layer): Linear(in_features=1, out_features=10, bias=True)
  (Output_layer): Linear(in_features=10, out_features=1, bias=False)
  (Sigmoid): Sigmoid()
)

In [29]:
class LSTM_Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=1, num_layers=1, batch_first=True)
        # self.output_layer = nn.Linear(10, 1, bias=False) # if hidden_size=10

    def forward(self, x):
        x = x.unsqueeze(-1)  # (batch, seq_len, input_size)
        lstm_out, _ = self.lstm(x)
        # output = self.output_layer(lstm_out) # if hidden_size=10
        output = lstm_out
        return output.squeeze(-1)  # Remove last dimension for consistency


In [30]:
LSTM_Network()

LSTM_Network(
  (lstm): LSTM(1, 1, batch_first=True)
)

In [31]:
def u_exact(J, x_0, t):
    return -(J * x_0) / (t[-1] - t[0])


def x_trial(t, N_x, x_0):
    A = x_0
    return (1 - t) * A + (1 - t) * t * N_x(t)


def u_trail(N_u, t, x_0, J):
    C = u_exact(J, x_0, t)
    return C + t * N_u(t)


def loss_1(t, J, models, x_0):
    N_x = models["N_x"]
    N_u = models["N_u"]

    t.requires_grad = True
    x_t = x_trial(t, N_x, x_0)
    dx_dt = torch.autograd.grad(x_t.sum(), t, create_graph=True)[0]
    u_tr = u_trail(N_u, t, x_0, J)

    G = ((J * dx_dt) - u_tr) ** 2

    return torch.sum(G)

In [32]:
# optimizer = optim.Adam(
#         [param for model_params in all_parms for param in model_params], lr=0.001
#     ) ## 30000 epochs, 2e-4 mse

# optimizer = optim.SGD(
#         [param for model_params in all_parms for param in model_params], lr=0.01, momentum=0.9
#     ) ## nan values

# optimizer = torch.optim.LBFGS(
#         [param for model_params in all_parms for param in model_params]
#     ) ## 20 epochs, variable mse
# optimizer = optim.RMSprop(
#         [param for model_params in all_parms for param in model_params],
#         lr=0.01,
#         alpha=0.99,
#         eps=1e-08,
#         weight_decay=0,
#         momentum=0.9,
#         centered=False,
#     ) ## 10000 epochs 1e-5 mse

# optimizer = optim.Adagrad(
#     [param for model_params in all_parms for param in model_params], lr=0.01
# )  ## 1000 epochs 1e-3 mse

# optimizer = optim.AdamW(
#         [param for model_params in all_parms for param in model_params], lr=0.01
#     ) ## 20000 epochs 2e-4 mse

# optimizer = optim.Adadelta(
#         [param for model_params in all_parms for param in model_params], lr=0.1
#     ) ## 5000 epochs 1e-2 mse

In [33]:
def Optimize(t, J, epochs, models, x_0):
    all_parms = [model.parameters() for model in models.values()]

    optimizer = optim.LBFGS(
        [param for model_params in all_parms for param in model_params]
    )

    def train():
        with torch.backends.cudnn.flags(enabled=False):
            optimizer.zero_grad()
            loss = loss_1(t, J, models, x_0)
            loss.backward()
        return loss

    for i in tqdm(range(epochs)):
        optimizer.step(train)

In [34]:
def MSE_Calculate(t, J, models, x_0):
    N_u = models["N_u"]
    with torch.no_grad():
        u_tr = u_trail(N_u, t, x_0, J).detach().cpu()
        u_ex = u_exact(J, x_0, t).detach().cpu()

    print(u_tr)
    print(u_ex)
    mse = torch.mean((u_tr - u_ex) ** 2)
    print(f"MSE: {mse}")

In [57]:
EPOCHS = 10
J = torch.tensor(20).to(device=device)  # J = 20
x_0 = torch.tensor(10).to(device=device)  # x_0 = 10

### u_exact = -J*x_0 = -200

T = torch.linspace(start=0, end=1, steps=100).unsqueeze(1).to(device=device)

# # Create models
# N_u = NeuralNetwork().to(device=device)
# N_x = NeuralNetwork().to(device=device)

# # Create LSTM models
N_u = LSTM_Network().to(device=device)
N_x = LSTM_Network().to(device=device)

models = {
    "N_u": N_u,
    "N_x": N_x,
}

Optimize(T, J, EPOCHS, models, x_0)
MSE_Calculate(T, J, models, x_0)


100%|██████████| 10/10 [00:00<00:00, 24.79it/s]

tensor([[-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200.0000],
        [-200

In [36]:
T = torch.linspace(start=0, end=1, steps=10).unsqueeze(1).to(device=device)
print(T.shape)
print(T.unsqueeze(-1).shape)

torch.Size([10, 1])
torch.Size([10, 1, 1])


In [37]:
##----> This cell is for exporting the model to onnx format

# # import torch

# # model = LSTM_Network().to(device)
# # dummy_input = torch.randn(5, 10).to(device)  # example input
# # torch.onnx.export(
# #     model,
# #     dummy_input,
# #     "lstm_model.onnx",
# #     input_names=["input"],
# #     output_names=["output"],
# #     dynamic_axes={
# #         "input": {0: "batch_size", 1: "seq_len"},
# #         "output": {0: "batch_size", 1: "seq_len"},
# #     },
# # )